In [3]:
import os
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
def create_columnar_database(host, user, password, port=5432):
    """Create database with separate tables per sensor type"""
    
    # Create database
    conn = psycopg2.connect(host=host, database="postgres", user=user, password=password, port=port)
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()
    
    cur.execute("SELECT 1 FROM pg_database WHERE datname='ambient_sensors_columnar'")
    if not cur.fetchone():
        cur.execute("CREATE DATABASE ambient_sensors_columnar")
        print("Columnar database created!")
    
    cur.close()
    conn.close()
    
    # Connect and create tables
    conn = psycopg2.connect(host=host, database="ambient_sensors_columnar", user=user, password=password, port=port)
    cur = conn.cursor()
    
    cur.execute("CREATE EXTENSION IF NOT EXISTS timescaledb;")

    # Devices table
    cur.execute("""
    CREATE TABLE IF NOT EXISTS devices (
        device_id VARCHAR(100) PRIMARY KEY,
        device_name VARCHAR(200),
        location VARCHAR(200),
        firmware_version VARCHAR(50),
        installed_date TIMESTAMPTZ DEFAULT NOW()
    );
""")
    
    # Sensors table with metadata JSONB column
    cur.execute("""
        CREATE TABLE IF NOT EXISTS sensors (
            sensor_id SERIAL PRIMARY KEY,
            device_id VARCHAR(100) NOT NULL,
            sensor_type VARCHAR(50) NOT NULL,
            location VARCHAR(200),
            metadata JSONB,
            installed_date TIMESTAMPTZ DEFAULT NOW()
        );
    """)
    
    # SCD30 measurements
    cur.execute("""
        CREATE TABLE IF NOT EXISTS scd30_measurements (
            time TIMESTAMPTZ NOT NULL,
            sensor_id INTEGER REFERENCES sensors(sensor_id),
            co2 FLOAT,
            temperature FLOAT,
            humidity FLOAT
        );
    """)
    
    # BMP280 measurements
    cur.execute("""
        CREATE TABLE IF NOT EXISTS bmp280_measurements (
            time TIMESTAMPTZ NOT NULL,
            sensor_id INTEGER REFERENCES sensors(sensor_id),
            pressure FLOAT,
            temperature FLOAT,
            humidity FLOAT
        );
    """)
    
    # Create hypertables
    for table in ['scd30_measurements', 'bmp280_measurements']:
        cur.execute(f"""
            SELECT 1 FROM timescaledb_information.hypertables 
            WHERE hypertable_name = '{table}'
        """)
        if not cur.fetchone():
            cur.execute(f"SELECT create_hypertable('{table}', 'time');")
    
    conn.commit()
    cur.close()
    conn.close()
    print("Columnar schema created!")

In [29]:
create_columnar_database(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    port=5432
)

Columnar database created!
Columnar schema created!


In [26]:
def create_flexible_database(host, user, password, port=5432):
    """Create database with flexible metric_type structure"""
    
    # Create database
    conn = psycopg2.connect(host=host, database="postgres", user=user, password=password, port=port)
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()
    
    cur.execute("SELECT 1 FROM pg_database WHERE datname='ambient_sensors_flexible'")
    if not cur.fetchone():
        cur.execute("CREATE DATABASE ambient_sensors_flexible")
        print("Flexible database created!")
    
    cur.close()
    conn.close()
    
    # Connect and create tables
    conn = psycopg2.connect(host=host, database="ambient_sensors_flexible", user=user, password=password, port=port)
    cur = conn.cursor()
    
    cur.execute("CREATE EXTENSION IF NOT EXISTS timescaledb;")

   # Devices table
    cur.execute("""
    CREATE TABLE IF NOT EXISTS devices (
        device_id VARCHAR(100) PRIMARY KEY,
        device_name VARCHAR(200),
        location VARCHAR(200),
        firmware_version VARCHAR(50),
        installed_date TIMESTAMPTZ DEFAULT NOW()
    );
""")

    # Sensors table with metadata JSONB column
    cur.execute("""
        CREATE TABLE IF NOT EXISTS sensors (
            sensor_id SERIAL PRIMARY KEY,
            device_id VARCHAR(100) NOT NULL,
            sensor_type VARCHAR(50) NOT NULL,
            location VARCHAR(200),
            metadata JSONB,
            installed_date TIMESTAMPTZ DEFAULT NOW()
        );
    """)
    
    # Flexible measurements
    cur.execute("""
        CREATE TABLE IF NOT EXISTS measurements (
            time TIMESTAMPTZ NOT NULL,
            sensor_id INTEGER REFERENCES sensors(sensor_id),
            metric_type VARCHAR(50) NOT NULL,
            value FLOAT
        );
    """)
    
    # Create hypertable
    cur.execute("""
        SELECT 1 FROM timescaledb_information.hypertables 
        WHERE hypertable_name = 'measurements'
    """)
    if not cur.fetchone():
        cur.execute("SELECT create_hypertable('measurements', 'time');")
    
    conn.commit()
    cur.close()
    conn.close()
    print("Flexible schema created!")

In [27]:
create_flexible_database(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    port=5432
)

Flexible database created!
Flexible schema created!


In [4]:
def delete_database(database_name):
    # Connect to postgres database instead
    try:
        conn = psycopg2.connect(
            host=os.getenv("DB_HOST"),
            database="postgres",
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            port=5432
        )
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

        cur = conn.cursor()

        cur.execute(f"""
            SELECT pg_terminate_backend(pid) 
            FROM pg_stat_activity 
            WHERE datname = '{database_name}' AND pid <> pg_backend_pid();
        """)

        cur.execute(f"DROP DATABASE {database_name};")

        cur.close()
        conn.close()
        print("Connection successful!")
    except Exception as e:
        print(f"Connection failed: {e}")    

In [ ]:
#delete_database("ambient_sensors_columnar")
#delete_database("ambient_sensors_flexible")

Connection successful!
Connection successful!


In [28]:
conn.close()

In [ ]:
import datetime

In [20]:
datetime.datetime.fromtimestamp(1761216744, tz=datetime.timezone.utc)

datetime.datetime(2025, 10, 23, 10, 52, 24, tzinfo=datetime.timezone.utc)